In [36]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, dotenv_values
import os
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


env_path = '.env'
env_vars = dotenv_values(env_path)
for key in env_vars:
    if key in os.environ:
        del os.environ[key]

# Explicitly reload the .env file
load_dotenv(env_path)

llm = ChatOpenAI(model="gpt-4o-mini")

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://github.com/foxyberry/work-doc/blob/main/keycloak/KO_KR.md",),
)

docs = loader.load()

len(docs[0].page_content)

4080

In [35]:
docs[0].page_content

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nwork-doc/ci/cd/automating-docker-builds-ko.kr.md at main · foxyberry/work-doc · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\nActions\n        Automate any workflow\n      \n\n\n\n\n\n\n\nPackages\n        Host and manage packages\n      \n\n\n\n\n\n\n\nSecurity\n        Find and fix vulnerabilities\n      \n\n\n\n\n\n\n\nCodespaces\n        Instant dev environments\n      \n\n\n\n\n\n\n\nGitHub Copilot\n        Write better code with AI\n      \n\n\n\n\n\n\n\nCode review\n        Manage code changes\n      \n\n\n\n\n\n\n\n

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input" : "How can I use Keycloak as Authentication Server?"})



To use Keycloak as an Authentication Server, you need to set it up by installing Keycloak and configuring a realm for your application. After that, you can create clients and users within that realm, and integrate your application by using the appropriate Keycloak adapters or REST APIs for authentication and authorization. Finally, configure your application to redirect to Keycloak for login and handle tokens for user sessions.
page_content='work-doc/keycloak/KO_KR.md at main · foxyberry/work-doc · GitHub













































Skip to content












Navigation Menu

Toggle navigation









 








            Sign in
          








        Product
        












Actions
        Automate any workflow
      







Packages
        Host and manage packages
      







Security
        Find and fix vulnerabilities
      







Codespaces
        Instant dev environments
      







GitHub Copilot
        Write better code with AI
      






In [43]:
print(response["answer"])
print()
for document in response["context"]:
    print(document)
    print()


To use Keycloak as an Authentication Server, you need to set it up by installing Keycloak and configuring a realm for your application. After that, you can create clients and users within that realm, and integrate your application by using the appropriate Keycloak adapters or REST APIs for authentication and authorization. Finally, configure your application to redirect to Keycloak for login and handle tokens for user sessions.

page_content='work-doc/keycloak/KO_KR.md at main · foxyberry/work-doc · GitHub













































Skip to content












Navigation Menu

Toggle navigation









 








            Sign in
          








        Product
        












Actions
        Automate any workflow
      







Packages
        Host and manage packages
      







Security
        Find and fix vulnerabilities
      







Codespaces
        Instant dev environments
      







GitHub Copilot
        Write better code with AI
      



